In [1]:
%load_ext autoreload
%autoreload 2

import plotly.io as pio
pio.renderers.default = 'iframe'

In [2]:
import pandas as pd
import plotly.express as px
import numpy as np
import pandas as pd
import re
from datetime import timedelta
import plotly.express as px

from op_analytics.datasources.defillama.dataaccess import DefiLlama

import urllib3
import warnings
pd.set_option('display.float_format', lambda x: '%.3f' % x)
urllib3.disable_warnings()
warnings.filterwarnings("ignore")

In [44]:

mapping = {
    "Dexes": "Dexes",
    "Liquidity manager": "Yield",
    "Derivatives": "Derivatives",
    "Yield Aggregator": "Yield",
    "Indexes": "Yield",
    "Bridge": "Bridge",
    "Leveraged Farming": "Yield",
    "Cross Chain": "Trading",
    "CDP": "Lending",
    "Farm": "Yield",
    "Options": "Derivatives",
    "DCA Tools": "Trading",
    "Services": "TradFi/Fintech",
    "Chain": "TradFi/Fintech",
    "Privacy": "TradFi/Fintech",
    "RWA": "TradFi/Fintech",
    "Payments": "TradFi/Fintech",
    "Launchpad": "TradFi/Fintech",
    "Synthetics": "Derivatives",
    "SoFi": "TradFi/Fintech",
    "Prediction Market": "Trading",
    "Token Locker": "Yield",
    "Yield Lottery": "Yield",
    "Algo-Stables": "Stablecoins",
    "DEX Aggregator": "Dexes",
    "Liquid Restaking": "Liquid Staking/Restaking",
    "Governance Incentives": "Gov",
    "Restaking": "Liquid Staking/Restaking",
    "Liquid Staking": "Liquid Staking/Restaking",
    "Uncollateralized Lending": "Lending",
    "Managed Token Pools": "Trading",
    "Insurance": "TradFi/Fintech",
    "NFT Marketplace": "Trading",
    "NFT Lending": "Lending",
    "Options Vault": "Derivatives",
    "NftFi": "Trading",
    "Basis Trading": "Yield",
    "Bug Bounty": "TradFi/Fintech",
    "OTC Marketplace": "Trading",
    "Reserve Currency": "Stablecoins",
    "Gaming": "Other",
    "AI Agents": "TradFi/Fintech",
    "Treasury Manager": "TradFi/Fintech",
    "CDP Manager": "Lending",
    "Decentralized Stablecoin": "Stablecoins",
    "Restaked BTC": "Liquid Staking/Restaking",
    "RWA Lending": "Lending",
    "Staking Pool": "Liquid Staking/Restaking",
    "CeDeFi": "TradFi/Fintech",
    "Staking": "Liquid Staking/Restaking",
    "Oracle": "Other",
    "Ponzi": "Other",
    "Anchor BTC": "Other",
    "Decentralized BTC": "Other",
    "CEX": "Other",
    "Lending": "Lending"
}


In [45]:
from op_analytics.coreutils.duckdb_inmem.client import init_client
from op_analytics.coreutils.duckdb_inmem.localcopy import dump_local_copy, load_local_copy
from op_analytics.datasources.defillama.dataaccess import DefiLlama


In [5]:

# ctx = init_client()
# client = ctx.client
from google.cloud import bigquery
from google.oauth2 import service_account

# tvl_view = DefiLlama.YIELD_POOLS_HISTORICAL.read(
#     min_date="2021-01-01",
# )

# # Process protocol TVL
# df_yield_pool = client.sql(f"""
#     SELECT
#         *
#     FROM {tvl_view}
# """).pl().to_pandas()

In [6]:
client = bigquery.Client()

In [ ]:
query_job = client.query(query)

results = query_job.result()

df = results.to_dataframe()

In [ ]:
# SELECT * FROM `oplabs-tools-data.temp.defillama_tvl_breakdown_enriched` 
# WHERE chain = 'Base'
#   AND parent_protocol = 'aave'
# AND dt = '2025-01-01'
# LIMIT 1000

In [14]:
!uv pip install db-dtypes

Using Python 3.12.7 environment at /Users/chuck/codebase/op-analytics/.venv
Resolved 9 packages in 14.82s                                        
Prepared 1 package in 1.49s                                              
Installed 1 package in 7ms                                  
 + db-dtypes==1.4.1


In [105]:
df = client.query(
f"""
SELECT 
    b.*,
    m.provider_entity,
    m.provider_entity_w_superchain
FROM oplabs-tools-data.temp.defillama_tvl_breakdown_filtered b
LEFT JOIN oplabs-tools-data.views.all_chains_metadata m
    ON LOWER(b.chain) = LOWER(m.chain)
WHERE dt = '2025-02-06'
""").result().to_dataframe()

In [106]:
df["protocol_category_mapped"] = df["protocol_category"].map(mapping, na_action="ignore")
df.loc[df["protocol_category_mapped"].isna(), "protocol_category_mapped"] = df["protocol_category"]


In [107]:
# df = df[(df.to_filter_out == 0)]

In [108]:
df["chain_agg"] = df.chain
df.loc[(df.provider_entity_w_superchain == "Arbitrum: Orbit"), "chain_agg"] = "Arbitrum Eco"
df.loc[(df.provider_entity_w_superchain == "Polygon: CDK"), "chain_agg"] = "Polygon Eco"
df.loc[(df.provider_entity_w_superchain == "ZKSync: ZK Stack"), "chain_agg"] = "zkSync Eco"
# df.loc[(df.provider_entity_w_superchain == "Optimism: Superchain"), "chain_agg"] = "Superchain"


In [135]:

chains_list = [
    "Arbitrum Eco",
    "Polygon Eco", 
    "ZkSync Eco",
    "Sui",
    "Solana",
    "Base",
    "Optimism",
    "Ink",
    "World Chain",
    "Soneium",
    "Ethereum"
    
]

category_list = [
    "Dexs",
    "Lending",
    "Yield",
    "Liquid Staking/Restaking",
    "Derivatives"
    
    
]

In [136]:
df_pivot = df[
    df.chain_agg.isin(chains_list)
    & (df.protocol_category_mapped.isin(category_list))
].pivot_table(
    index="protocol_category_mapped",
    columns="chain_agg",
    values="app_token_tvl_usd",
    aggfunc="sum"
).reset_index()

In [139]:
df_pivot

chain_agg,protocol_category_mapped,Arbitrum Eco,Base,Ethereum,Ink,Optimism,Polygon Eco,Solana,Soneium,Sui,World Chain
0,Derivatives,616543339.602,152155064.657,564015165.777,NaN,143740875.357,4242070.384,2806773373.604,251050.750,18163202.616,NaN
1,Dexs,570320141.816,1608755758.081,7069369208.000,4715232.208,124053457.577,232943839.287,2383018493.409,9493898.732,375583722.573,1982618.577
2,Lending,1179398788.374,1144468206.971,30482911673.166,NaN,204350942.678,364879150.361,2725372103.527,6226045.110,1035122118.908,NaN
3,Liquid Staking/Restaking,128330125.515,21710252.797,63605680202.766,NaN,11202432.521,NaN,10611769988.558,2175085.817,535236458.604,NaN
4,Yield,470848016.201,606039320.654,21640796773.212,NaN,123018191.490,68062795.018,747773684.380,1300105.382,88749253.356,43.270


In [141]:
2383018493.409	 - 7069369208.000

-4686350714.591

In [126]:
df_pivot_stables = df[
    df.chain_agg.isin(chains_list)
    & (df.token_category == "Stablecoins")
    & (df.protocol_category != "Bridge")
].pivot_table(
    index="token_category",
    columns="chain_agg",
    values="app_token_tvl_usd",
    aggfunc="sum"
).reset_index()

In [127]:
df_pivot_final = pd.concat([df_pivot, df_pivot_stables])

In [128]:
df_pivot_final["protocol_category_mapped"].fillna("Stablecoins", inplace=True)

In [134]:
df_pivot_final[[
    "protocol_category_mapped",
    "Base",
    "Optimism",	
    "Soneium",
    "Ink",
    "World Chain",
    "Solana",
    "Arbitrum Eco",
    "Sui",
    "Polygon Eco"]].to_csv("raw_tvl_chain_competitor_feb_6_2025.csv", index=False)